In [ ]:
#!gdown --id 1iUJlEkD0luAy3ri8BxdfXHoZliZPD6ZM

#### Download files from drive
!gdown --id "1zAHe0nRAxzTsrBm3pNfqjQZeHd-uKgTd"
!unzip -q "/content/altegrad-2020.zip"
!gdown --id 11BK192XungSfpZVrgQo9Hi4tOeOcLdoR
#### install libraries
import nltk
nltk.download("stopwords")
!pip  install -q keybert[flair]

# Clean and concatenate author abstracts : 

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ast 
from nltk.corpus import stopwords 
import json
import re



f = open("author_papers.txt","r")
papers_set = set()
dic_aut_paper = {}
for l in f:
    auth_paps = [paper_id.strip() for paper_id in l.split(":")[1].replace("[","").replace("]","").replace("\n","").replace("\'","").replace("\"","").split(",")]
    dic_aut_paper[l.split(":")[0]] = auth_paps
f.close()


dic_paper_aut = {}

for (k,v) in dic_aut_paper.items():
    for id_p in v:
        if id_p in dic_paper_aut.keys():
            dic_paper_aut[id_p] += [k]
        else :
            dic_paper_aut[id_p] = [k]

print(len(dic_paper_aut))
print(len(dic_aut_paper))


1185241
231239


In [ ]:
import tqdm.notebook as tq
#for i in tq.tqdm(...):

pattern = re.compile(r'(,){2,}')
stop_words = set(stopwords.words('english')) 

f = open("abstracts.txt","r")

# loads the inverted abstracts and stores them as id-abstracts in a dictionary dic and in a folder fw
dim = 256
dic = {}

visited_papers = []
for l in tq.tqdm(f):
    
    if(l=="\n"):
        continue
    id = l.split("----")[0]
    inv = "".join(l.split("----")[1:])
    res = ast.literal_eval(inv) 
    abstract =[ "" for i in range(res["IndexLength"])]
    inv_indx=  res["InvertedIndex"]

    for i in inv_indx:
        if i.isalpha() and i not in stop_words:
            for j in inv_indx[i]:
                abstract[j] = i.lower()
    abstract = re.sub(pattern, ',', ",".join(abstract))
    ### get the authors who wrote this paper
    auth_ids = dic_paper_aut[id]
    ## update the abstract for all the authors who took part in the paper :D 
    for auth_id in auth_ids:

      if auth_id in dic.keys():
          dic[auth_id] += abstract
      else:
          dic[auth_id] = abstract

print(len(dic))

with open('abstracts_processed_concat.txt', 'w') as file:
     file.write(json.dumps(dic)) 

import json 
  
# reading the data from the file 
with open('abstracts_processed_concat.txt') as f: 
    data = f.read() 
  
print("Data type before reconstruction : ", type(data)) 
      
# reconstructing the data as a dictionary 
js = json.loads(data) 
  
print("Data type after reconstruction : ", type(js)) 

print(len(js))


228738
Data type before reconstruction :  <class 'str'>
Data type after reconstruction :  <class 'dict'>
228738


In [ ]:
import networkx as nx
from itertools import chain 

"""Here we correct for the authors who don't have an abstract

We give the the set of abstracts of their closest valid neighbour """

def one_neighbour(G,auth):
  auth = int(auth)
  nbrs = list(G.neighbors(auth))
  nbrs = set([str(i) for i in nbrs])
  valid_nbrs = nbrs.intersection(dic.keys())
  valid_nbrs = list(valid_nbrs)

  hop_cnt = 0

  while len(valid_nbrs)==0:
      #print("checking scnd neighbours")
      hop_cnt +=1
      scnd_nghbr = [G.neighbors(int(nbr)) for nbr in nbrs]
      ## flatten list
      nbrs = list(chain(*scnd_nghbr))
      nbrs = set([str(i) for i in nbrs])
      valid_nbrs = nbrs.intersection(dic.keys())
      valid_nbrs = list(valid_nbrs)

      if hop_cnt ==3:
        print("this one is really lonely")
        return 'orphan'

  return str(valid_nbrs[0])

In [ ]:
# load the graph    
G = nx.read_edgelist('collaboration_network.edgelist', delimiter=' ', nodetype=int)
## get none found authors
lost_auths = set(list(dic_aut_paper)) - set(list(dic))
lost_auths = list(lost_auths)
## get one nghbr with an abstract for each author
lost_auths_nghbr = {auth:one_neighbour(G,auth) for auth in lost_auths}

### put the abstract of the neighbour for this author
for i,auth in enumerate(lost_auths):
  nbr = lost_auths_nghbr[auth]
  dic[auth] = dic[nbr]

# Keyword extraction with KeyBERT:

In [ ]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
import itertools
from time import time 
import pickle

CHUNK_SIZE= 10000
docs = list(dic.values())

## Batch the documents for faster processing

docs_chunks = [docs[x:x+CHUNK_SIZE] for x in range(0,len(docs),CHUNK_SIZE)]

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cuda")
model_mini = KeyBERT(model=sentence_model)

keywords = []
start = time()
for chunk in tq.tqdm(docs_chunks):
  
  keyword_chunk =model_mini._extract_keywords_multiple_docs(chunk,top_n=30,
                                                            keyphrase_ngram_range=(1, 1),min_df=10)
  keywords.append(keyword_chunk)

## flatten our list
keywords = list(itertools.chain(*keywords))
### fix small error due to uknown token
for i,value in enumerate(keywords) :
  if value == ['None Found']:
    keywords[i] = [(1,'Science '),(2,'Paper')]
#### keywords come with their score
### in this line we discard the scores


In [ ]:
abstract_keywords = list(map(lambda x: ' '.join([str(i) for i,j in x]), keywords))
keywords_dict = {docID:words for docID,words in zip(list(dic.keys()),abstract_keywords)}
pd_keywds = pd.DataFrame(list(keywords_dict.values()),columns=['key_words'])
pd_keywds['authorID'] = list(keywords_dict.keys())
pd_keywds.set_index("authorID",inplace=True)
pd_keywds.head()

,key_words
authorID,
2108281488,associated components exhibits mechanisms cont...
2595221017,transmission evolved accessing allocating info...
2097270384,patterns entails web correlation connectivity ...
2147216009,spaces assumptions framework proposed inclusio...
2401159163,generalized nodes terms inventories discussed ...


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
pd_keywds.to_csv("auth_keywds_1.csv",index=False)
!cp "/content/auth_keywds_1.csv" '/gdrive/My Drive/'

# Generate embeddings with BERT :


In [ ]:
import pickle
!pip install -q  -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens',device='cuda')

In [ ]:
from itertools import chain
import random

CHUNK_SIZE = 10000

keywords = list(keywords_dict.values())
#random.shuffle(keywords)

keywords = list(keywords_dict.values())
for i,value in enumerate(keywords) :

  if value == ['None Found']:
    keywords[i] = 'None found'

keywords_chunks = [keywords[x:x+CHUNK_SIZE] for x in range(0,len(keywords),CHUNK_SIZE)]
i=0
auth_embeds = []
for chunk in keywords_chunks:
  i+=1
  print(i)
  chunk_embeds = model.encode(chunk)
  auth_embeds.append(chunk_embeds)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [ ]:
### flatten list
author_embeds = list(chain(*auth_embeds))
auth_embed_dict = {authID:embed  for authID,embed  in zip(list(keywords_dict.keys()),author_embeds)}

embed_shape = author_embeds[0].shape[0]
print("Embedding dimension=",embed_shape)
columns = ['auth_embed_'+str(i) for i in range(embed_shape)]
auth_embeds  = pd.DataFrame(list(auth_embed_dict.values()),columns=columns)
auth_embeds ['authorID'] = list(keywords_dict.keys())
auth_embeds.set_index("authorID",inplace=True)
auth_embeds.head()

Embedding dimension= 768


,auth_embed_0,auth_embed_1,auth_embed_2,auth_embed_3,auth_embed_4,auth_embed_5,auth_embed_6,auth_embed_7,auth_embed_8,auth_embed_9,auth_embed_10,auth_embed_11,auth_embed_12,auth_embed_13,auth_embed_14,auth_embed_15,auth_embed_16,auth_embed_17,auth_embed_18,auth_embed_19,auth_embed_20,auth_embed_21,auth_embed_22,auth_embed_23,auth_embed_24,auth_embed_25,auth_embed_26,auth_embed_27,auth_embed_28,auth_embed_29,auth_embed_30,auth_embed_31,auth_embed_32,auth_embed_33,auth_embed_34,auth_embed_35,auth_embed_36,auth_embed_37,auth_embed_38,auth_embed_39,...,auth_embed_728,auth_embed_729,auth_embed_730,auth_embed_731,auth_embed_732,auth_embed_733,auth_embed_734,auth_embed_735,auth_embed_736,auth_embed_737,auth_embed_738,auth_embed_739,auth_embed_740,auth_embed_741,auth_embed_742,auth_embed_743,auth_embed_744,auth_embed_745,auth_embed_746,auth_embed_747,auth_embed_748,auth_embed_749,auth_embed_750,auth_embed_751,auth_embed_752,auth_embed_753,auth_embed_754,auth_embed_755,auth_embed_756,auth_embed_757,auth_embed_758,auth_embed_759,auth_embed_760,auth_embed_761,auth_embed_762,auth_embed_763,auth_embed_764,auth_embed_765,auth_embed_766,auth_embed_767
authorID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2108281488,-0.173046,0.393209,1.831028,0.371120,0.242482,-0.495410,-0.726133,0.042257,0.460925,-0.613946,-0.336119,0.582738,-0.155096,0.496062,-0.131147,0.084022,-1.124881,-0.140932,-0.074926,0.084837,-0.253605,-0.096266,-0.329991,0.012299,0.851879,-0.130658,-0.248079,0.121888,-0.085081,-0.159354,-0.362353,0.105696,-0.159798,-0.924182,-0.654803,0.344883,0.286669,-0.077704,0.173878,0.154905,...,-0.598010,-0.826115,1.719797,-0.761064,0.179567,0.640447,0.892722,0.733161,0.066956,0.232464,-0.026667,-0.693749,-0.937065,0.354888,-0.028239,0.696455,-0.446061,0.024826,0.676387,0.156982,-0.157734,0.097707,0.034603,-0.720088,0.414393,0.288075,-0.383244,0.190391,-0.330003,0.187892,-0.281891,-0.852650,-0.476081,-0.633179,-0.690824,-0.123956,0.156236,-0.852114,-1.495437,0.033236
2595221017,-0.197502,-0.064884,1.619864,-0.014593,0.792456,-0.864574,-0.037857,-0.165942,0.276517,-0.555294,-0.631644,0.116088,-0.433079,0.563198,-0.049755,0.489740,-0.389780,0.468629,-0.035671,-0.259494,-0.996263,0.508401,0.713443,-0.176310,0.682409,0.020635,-0.100275,0.634538,-0.272070,0.307866,0.021029,-0.364702,0.281459,-0.348118,-0.306025,0.015886,0.131905,-0.432933,0.109667,-0.089357,...,-0.676947,-0.555963,-0.249646,-0.447208,-0.038728,0.235958,0.978269,0.097761,0.015489,-0.308050,-0.176071,-0.521514,-0.589157,-0.882483,-0.884800,0.661087,-0.235123,-0.218745,0.215607,0.418753,-0.721329,0.268213,0.336432,-0.037905,0.255193,0.882667,0.473685,0.300780,-0.686156,-0.092589,-0.650778,-1.114661,-0.039282,-0.118444,-0.244923,-0.153532,0.087447,0.042332,-0.823323,0.500721
2097270384,-0.192959,0.001563,1.654724,0.365408,0.396445,-0.608603,-0.352275,0.338049,-0.120389,-0.371518,-0.578296,0.462431,-0.789411,0.935133,0.549331,0.293660,-1.135520,0.178950,-0.177027,0.015445,-0.037963,-0.215782,-0.749321,0.028822,0.997122,0.002455,-0.392629,-0.169499,-0.804588,-0.054057,-0.848935,0.142752,0.187890,-0.549465,-0.328108,0.217560,-0.149524,-0.380250,0.418266,-0.326568,...,-0.746994,-0.469232,1.150694,-0.748614,0.096924,0.493252,-0.018203,-0.224732,0.192400,0.102610,-0.700352,-0.320948,-0.523485,-0.306408,-0.087058,0.075169,-0.273571,0.307482,1.067765,0.241145,-0.763631,0.658328,-0.081867,-1.038291,0.137976,0.870237,-0.258353,0.226808,-0.747607,-0.322230,-0.310240,-0.333399,-0.976173,0.048537,-0.065136,-0.617560,-0.023539,-0.869432,-0.564193,0.158580
2147216009,-0.985928,0.539725,0.853956,0.542073,0.184017,0.101193,-0.681880,-0.936817,0.359112,-0.898177,-0.627917,0.304435,0.010130,-0.132877,0.081373,0.354780,-0.613645,-0.981481,-0.308267,0.090582,0.293163,0.272516,-0.246858,-0.738789,0.771048,0.276306,-0.216015,0.419478,-1.267514,0.540901,-0.663147,0.165398,-0.097579,-0.501034,0.048533,0.016264,0.941655,-0.000413,0.037218,0.035471,...,-0.692499,-0.722468,0.3

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
auth_embeds.to_csv("author_embeds_bert768.csv",index=False)
!cp "/content/author_embeds_bert768.csv" "/gdrive/MyDrive/"

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
